# 5. Generating Realistic Stimuli with FilterNet

In the previous chapters we created biophysically detailed and point neuron toy models of the Mouse VISp L4 and stimulated them with LGN-like cells with firing rates using a Poisson distribution. This can provide insightful information about our model. But if we want to build a model that can more accurately mimic the behavior of a mouse visual system we will need to start providing more realistic stimuli - stimuli that would be generated by the retina and thalamus.

This is where the FilterNet module of BMTK comes into play. With FilterNet we can project different types of visual stimuli onto receptive fields of "filter" type neurons. Using a linear-nonlinear Poisson model, FilterNet will generate spike trains for each neuron based on the temporal and spatial dynamics of the visual stimuli. For modeling the LGN receptive fields, we will use 2D Gaussians for simplicity. To gain a more in depth intuition about the spatial and temporal filters, see [Modeling the visual field (with FilterNet): Units](https://alleninstitute.github.io/bmtk/tutorial_filter_models.html#Units).

![](../images/bmtk_lnp.png)

For this tutorial we will use FilterNet to generate LGN response spike trains to different types of visual stimuli, then use the feedforward connections from our previous model to deliver these inputs to our PointNet (or BioNet) L4 network.

![](../images/filternet_to_pointnet_workflow.png)

After that we will build a more advanced model of the LGN which will come in handy later as we try to recreate the full mouse V1 model.


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
import nest

from bmtk.utils.create_environment import create_environment
from bmtk.simulator import filternet
from bmtk.analyzer.spike_trains import plot_raster
from bmtk.simulator import pointnet

<div class="alert alert-block alert-info">
<b>Tip</b>: There is a small amount of biologically relevant randomness built in to the models and simulation, so everytime the notebook is run it is expected to get different results. However should you want to match the same results with the "completed" version of the tutorials please use the rng seed below:
</div>

In [ ]:
np.random.seed(100)

## Importing the PointNet Model

The first step is to copy the previous chapter's LGN and L4 model files, then create the configuration files and scripts for running a FilterNet simulation. As before we have already taken care of this for you. However if you wanted to do so from scratch:

```bash
$ cp -r ../Ch4_pointnet/network/* network/
$ cp -r ../Ch4_pointnet/components/* components/

# Create configs and run scripts for FilterNet
$ python -m bmtk.utils.create_environment            \
           --config-file config.filternet_flash.json \
           --network-dir network                     \
           --network-filter lgn_node                 \
           --output-dir inputs                       \
           --tstop 3000.0                            \
           --overwrite                               \
           filternet .
```

In [ ]:
from bmtk.utils.create_environment import create_environment

create_environment(
    'filternet',
    base_dir='.',
    config_file='config.filternet_flash.json',
    network_dir='network',
    network_filter='lgn_node',     # used to filter out only the lgn_nodes.h5 and lgn_node_types.csv file
    output_dir='inputs',
    tstop=3000.0,
    # overwrite=True
)

#### Setting the stimuli

Now we must choose the stimuli, which we can do using the "**inputs**" section of _config.simulation_filternet.json_. Besides regular movies, there are a number of built-in stimuli type that FilterNet will create for us. We'll start with a <ins>Full-Field Flash</ins> stimulation.


```json
 {
   "inputs": {
     "full_field_flash": {
       "input_type": "movie",
       "module": "full_field_flash",
       "row_size": 240,
       "col_size": 120,
       "t_on": 1000.0,
       "t_off": 2000.0,
       "max_intensity": 1.0,
       "frame_rate": 1000.0
     }
   }
 }
```

* We set **input_type** to 'movie' and **module** to "full_field_flash" which will direct BMTK to the type of stimuli to create.

* We selected the array sizes representing the visual space (**row_size** and **col_size**) to be 240x120 and will continue using this resolution throughout this chapter. Note, however, that this a choice that users can adjust.

* Next we tell BMTK to turn on the stimuli at 1000 ms (**t_on**) and off at 2000 ms (**t_off**). The intensity of the flash is set by **max_intensity** which is positive for white, negative for black, and default 0 for a grey screen.


We also want to modify the "**outputs**" sections so that the spike-trains generated by FilterNet are saved to the _inputs/_ directory (since indeed we will be using these spikes as inputs for our L4 simulation).

```json
 {
   "output": {
     "output_dir": "./inputs",
     "log_file": "log.txt",
     "rates_h5": "rates.flash.h5",
     "spikes_file_csv": "spikes.flash.csv",
     "spikes_file": "spikes.flash.h5"
   }
 }
```


#### Running FilterNet

We can run filternet in the command line terminal using the _run_filternet.py_ script:

```python
$ python run_filternet.py config.filternet_flash.json
```

If we were dealing with a very large network we can even run it on multiple cores:
```python
$ mpirun -np <N> python run_filternet.py config.filternet_flash.json
```

or we can run it in the notebook:

In [ ]:
from bmtk.simulator import filternet

config = filternet.Config.from_json('config.filternet_flash.json')
config.build_env()

net = filternet.FilterNetwork.from_config(config)
sim = filternet.FilterSimulator.from_config(config, net)
sim.run()

We can plot the results of providing a full-field flash to our LGN cells:

In [ ]:
from bmtk.analyzer.spike_trains import plot_raster

_ = plot_raster(config_file='config.filternet_flash.json', group_by='model_template')

Remember that when we built our LGN cells we used two different types of cell-models: transient ON cells (*tON_TF8*) and transient OFF cells (*tOFF_TF8*). 

```python
lgn = NetworkBuilder('lgn')
x, y = get_coords_plane(50)
lgn.add_nodes(
    N=50,
    x=x,
    y=y,
    model_type='virtual',
    model_template='lgnmodel:tON_TF8',
    dynamics_params='tON_TF8.json',
    ei_type='e'
)
x, y = get_coords_plane(50)
lgn.add_nodes(
    N=50,
    x=x,
    y=y,
    model_type='virtual',
    model_template='lgnmodel:tOFF_TF8',
    dynamics_params='tOFF_TF8.json',
    ei_type='e'
)
```

From the results of our full-field flash we can clearly see how the two different types of cells react to changes in the stimulus.

### Rerunning the L4 model with full-field flash stimuli.


Now that we have a new set of stimuli for our LGN cells, rather than the simple Poisson spike trains we were previously using, it is a trivial matter to substitute in our full-field flash results. 

As before we will need to create a separate config file for our ```PointNet``` simulation, which we'll call *config.pointnet_flash.json*. We can copy and rename an existing one from the previous chapter, or using the ```create_environment``` function:

All we have to do is open *config.simulation_pointnet.py* and modify the "**inputs**" section so it is now using the new *spikes.flash.h5* file we created.


Then run PointNet as like before and plot the results.

In [ ]:
from bmtk.utils.create_environment import create_environment

create_environment(
    'pointnet',
    base_dir='.',
    config_file='config.pointnet_flash.json',
    network_dir='network',
    output_dir='output_flash',
    tstop=3000.0, 
    dt=0.1,
    spikes_inputs=[('lgn', 'inputs/spikes.flash.h5')],
    # overwrite=True
)


The important thing is that in the "inputs" section we make sure the *input_file* is pointing to the one we just created using ```FilterNet```, *./inputs/spikes.flash.h5*. 

Now we can run ```PointNet``` with our newly generated inputs and show the results.

In [ ]:
from bmtk.simulator import pointnet

configure = pointnet.Config.from_json('config.pointnet_flash.json')
configure.build_env()

network = pointnet.PointNetwork.from_config(configure)
sim = pointnet.PointSimulator.from_config(configure, network)
sim.run()

In [ ]:
from bmtk.analyzer.spike_trains import plot_raster

_ = plot_raster(config_file='config.pointnet_flash.json', group_by='model_name', show=False)

### View the l4 pointnet flash spike activity with VND

Launch VND by typing `vnd` on the Linux command line.

In the Visual Neuronal Dynamics window select, select menu item  **File : Open File with Edges**
and choose the file `Ch5_filternet/config.pointnet_flash.json'


In the Main tab, in the Represenations pane:

In the list of representations, after loading, the single, default selection is:

- Selected Neurons: all
- Coloring Method: Type
- Style: soma

Select menu item  **File : Add File with Spikes** and choose `Ch5_filternet/output_flash/spikes.h5`

In the Activity window, set

- Population:l4

Click **Update Selection**

set:
- Color: blue
- Sphere Scale: 10
- Sphere Resolution: 5
- Step: 1
- Time window: 10

If no sturctures are showing, you may need to select **Display : Reset View**.  (Or even, in the Main tab Representations pane,select the soma, "all" representation, then select **Display : Reset View**.)

Drag the time slider left and right, drag it back to the far left, then press the right-pointing play button to play through the series of spikes.  Change speed with the Speed slider.



Direct link to <a href="http://www.ks.uiuc.edu/~barryi/bmtk_allen_workshop_2022/ch5_pointnet_flash_spikes.mp4"> pointnet flash spikes video <a>.
    

### Running FilterNet with Brain-Observatory Stimuli

Besides full-field flashes, FilterNet can automatically generate all kinds of stimuli like [Drifting Gratings](https://alleninstitute.github.io/bmtk/filternet.html#grating) and [Looming Stimuli](https://alleninstitute.github.io/bmtk/filternet.html#looming).

But for even more advanced stimuli we may want to create our own movie. Currently FilterNet can use any numpy matrix file (.npy or npz) the user creates. When creating movie files we recommend the following rules:

* The array should be of dimensions TxRxC (time x row x column) - where RxC is the array size of the visual space, and each slice in time represents a different frame.


* The filters models used in this network were optimized using stimuli running at a 1000Hz frame rate which we'll continue using here.


* Movies should be grey-scaled, with floating-point pixel values between \[-1, +1\], but if the movie uses \[0, 255\] FilterNet will convert it automatically.


#### Allen Brain-Observatory

The [Allen Brain Observatory](https://observatory.brain-map.org/visualcoding/) provides a large and robust dataset of cell responses to different visual stimuli - and a good set of in-vivo data for which we can customize and compare our models to. Among the stimuli used in the Allen Brain Observatory includes static images of [natural scenes](https://observatory.brain-map.org/visualcoding/stimulus/natural_scenes) and [movies](https://observatory.brain-map.org/visualcoding/stimulus/natural_movies). We will now show how we can recreate these stimuli and use it in our own model.

Starting with natural scene images, we'll need to use the AllenSDK to download the 118 images then use them to create a movie. The following functions will help us:

<div class="alert alert-block alert-warning">
<b>Note:</b> Although what is being played is a series of static images, we still refer to the input as a "movie". And indeed the only difference between natural scenes and an actual movie is the number of times the screen changes per second.
</div>

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
from PIL import Image

try:
    from allensdk.core.brain_observatory_cache import BrainObservatoryCache
except ImportError:
    from bmtk.utils.brain_observatory.brain_observatory_cache import BrainObservatoryCache


def get_natural_scenes(output_dir='bob_images'):
    """Fetches the 118 Brain Obs natural scene images from the data, saves them in npy/npz format"""
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    boc = BrainObservatoryCache(manifest_file='boc/manifest.json')
    data_set = boc.get_ophys_experiment_data(501498760)
    scenes = data_set.get_stimulus_template('natural_scenes')

    for i in range(0, len(scenes)):
        scene = scenes[i]
        base_name = os.path.join(output_dir, 'scene.{:03d}.gray_{}x{}'.format(i, scene.shape[0], scene.shape[1]))
        plt.imsave('{}.png'.format(base_name), scene, cmap='gray')


def create_movie_natural_scenes(images, movie_path, image_dur=250.0,  gs_dur=500.0, res_row=120, res_col=240, fps=1000.0, compress=True):
    """Will create a movie of the given images - with a pre- and post- grey-screen of the same duration"""
    frames_per_image = int((image_dur/1000.0)*fps)
    frames_per_gs = int(int((gs_dur/1000.0)*fps))
    n_frames = frames_per_gs + len(images)*frames_per_image + frames_per_gs

    output_mat = np.zeros((n_frames, res_row, res_col), dtype=float)
    c_frame = frames_per_gs
    for img_path in images:
        pic = Image.open(img_path).convert('L')
        pic = pic.resize((res_col, res_row))
        pic_data = np.asarray(pic)
        pic_data = pic_data.astype(dtype=float) * 2.0 / 255.0 - 1.0
        output_mat[c_frame:(c_frame + frames_per_image), :, :] = pic_data

        c_frame += frames_per_image

    movies_dir = os.path.dirname(movie_path)
    if not os.path.exists(movies_dir):
        os.makedirs(movies_dir)
        
    if compress:
        np.savez(movie_path, output_mat)
    else:
        np.save(movie_path, output_mat)


def show_movie(movie_file, frames):
    """Helps visualize the movie"""
    movie_array = np.load(movie_file)
    if movie_file.endswith('npz'):
        # is file is saved in npz format instead of npy
        movie_array = movie_array['arr_0']
    
    fig, ax = plt.subplots(1, len(frames), figsize=(40, 5*len(frames)))

    for i, frame in enumerate(frames):
        ax[i].imshow(movie_array[frame, :, :], cmap='gray', vmin=-1.0, vmax=1.0)
        # ax[i].set_xticks([])
        ax[i].set_xticks([0])
        ax[i].set_xticklabels([frame],fontsize=20)

        ax[i].set_yticks([])

    ax[5].set_xlabel('Frame #', horizontalalignment='right', fontsize=20)
    plt.subplots_adjust(wspace=0, hspace=0)

    plt.show()

In [ ]:
if not os.path.exists('bob_images'):
    # Only need to download bob_images once
    get_natural_scenes()

# Fetch 8 images, create a movie of images including beginning and ending grey-screen
images = glob('bob_images/scene*.png')
images = np.random.choice(images, size=8, replace=False)
create_movie_natural_scenes(images, movie_path='movies/ns_movie.8images.npz')
show_movie(movie_file='movies/ns_movie.8images.npz', frames=range(0, 3000, 250))

Now that we have our natural-scenes (ns) movies we can, like above, use it to generate inputs for our ```PointNet``` l4 model simulation. First we just need to create the appropiate ```FilterNet``` and ```PointNet``` configuration files. We've already created them for you called *config.filternet_ns.json* and *config.pointnet_ns.json*, but feel free to create them yourselves - either by copying and modifying previous configs, or using the ```bmtk.utils.create_environment``` function either through the command line or in Jupyter.


For our ```FilterNet``` configuration file, *config.filternet_ns.json*, we just to update the "input" section so bmtk knows to use the movie/npy file we generated. Also we want to update the "output" so the rates and spikes files generated are given a more descriptive name (and won't overwrite previous results).

```json
  {
    "inputs": {
      "input_type": "movie",
      "module": "movie",
      "data_file": "./movies/ns_movie.8images.npz",
      "frame_rate": 1000.0        
    },
      
    "output": {
      "output_dir": "./inputs",
      "log_file": "log.txt",
      "rates_h5": "rates.ns_movie.h5",
      "spikes_file_csv": "spikes.ns_movie.csv",
      "spikes_file": "spikes.ns_movie.h5"
    }
  }
    
```

And run FilterNet as we have been doing.

In [ ]:
from bmtk.simulator import filternet

config = filternet.Config.from_json('config.filternet_ns.json')
config.build_env()

net = filternet.FilterNetwork.from_config(config)
sim = filternet.FilterSimulator.from_config(config, net)
sim.run()

In [ ]:
from bmtk.analyzer.spike_trains import plot_raster

_ = plot_raster(config_file='config.filternet_ns.json', group_by='model_template')

To run PointNet with the new FilterNet output just edit the *config.pointnet_ns.json* 's "input" section so the name of the spike trains files is now pointing to *inputs/spikes.natural_scenes.h5*.


In [ ]:
from bmtk.simulator import pointnet

configure = pointnet.Config.from_json('config.pointnet_ns.json')
configure.build_env()

network = pointnet.PointNetwork.from_config(configure)
sim = pointnet.PointSimulator.from_config(configure, network)
sim.run()

In [ ]:
from bmtk.analyzer.spike_trains import plot_raster

_ = plot_raster(config_file='config.pointnet_ns.json', group_by='model_name', show=False)

### View the l4 pointnet natural-scene spike activity with VND

  Animate the l4 spike activity with VND, following the same method as
earlier in this chapter.  Use model file `Ch5_filternet/config.pointnet_ns.json` and spike file `Ch5_filternet/output_ns/spikes.h5`.

Direct link to <a href="http://www.ks.uiuc.edu/~barryi/bmtk_allen_workshop_2022/ch5_pointnet_ns_spikes.mp4"> pointnet natural-scene (ns) spikes video <a>.



The Allen Brain-Observatory stimulus also includes actual movies, three clips taken from the 1958 Orson Welles film "Touch of Evil". Here, we generate two of the 30 second clips, "natural_movie_one" and "natural_movie_two".

The following function will help download the movies, convert them to 120x240 resolution, upscale the frame rate from 30 fps to 1000 fps, and save the npy files into the _movies/_ directory.

In [ ]:
def get_touchofevil_movies(output_dir='movies', res_row=120, res_col=240, fps=1000, compress=True):
    frame_conv = int(np.floor(fps/30.0))
    def convert_movie(name, movie):
        t, x, y = movie.shape
        n_frames = frame_conv * t
        movie_updated = np.zeros((n_frames, res_row, res_col), dtype=float)
        c_frame = 0
        for frame in range(t):
            # Resize resolution
            img = Image.fromarray(movie[frame, :, :], mode='L')
            img = img.resize((res_col, res_row))
            img_data = np.asarray(img)
            img_data = img_data.astype(dtype=float) * 2.0 / 255.0 -1.0

            # Upscale frame rate
            movie_updated[c_frame:(c_frame + frame_conv), :, :] = img_data
            c_frame += frame_conv

        if compress:
            np.savez('{}/{}.{}ms.{}x{}.npz'.format(output_dir, name, c_frame, res_row, res_col), movie_updated)
        else:
            np.save('{}/{}.{}ms.{}x{}.npy'.format(output_dir, name, c_frame, res_row, res_col), movie_updated)

    boc = BrainObservatoryCache(manifest_file='boc/manifest.json')
    data_set = boc.get_ophys_experiment_data(506248008)
    movie = data_set.get_stimulus_template('natural_movie_one')
    convert_movie('natural_movie_one', movie)

    movie = data_set.get_stimulus_template('natural_movie_two')
    convert_movie('natural_movie_two', movie)

In [ ]:
# get_touchofevil_movies()

If you want to run the LGN and L4 simulation with these movies remember to do the following:

1. In the FilterNet simulation config update the **input** movie path to the correct *npz file name.

2. Update FilterNet's "**output**" 'spikes_file' to a more appropiate name. And make sure to update PointNet's **input** section accordingly.

3. For Both FilterNet and PointNet update the "**run**" sections 'tstop' to 30,000 to simulate the entire clip.

### [OPTIONAL] Running the Simulations on a Biophysical (BioNet) model

So far we showed how to use the realistic stimuli generated by ```FilterNet``` as input for our GLIF/point-neuron model of our toy "l4" network. But the "full-field-flash" or "natural-scenes" spike-trains generated above can just as easily be used with the biophysical version of the "l4" model we generated in the previous chapters using ```BioNet```. It's just a matter of changing the "inputs" sections of one our our ```BioNet``` configs and replacing a randomized spike-trains files with one generated using ```FilterNet```.

**Note:** that the results of ```FilterNet``` simulations are heavily dependent on the placement of the individual filter-cells. If the "lgn" cells in the Biophysical models are different than in the GLIF models it's important to rerun ```FilterNet``` with the differen "lgn_nodes.h5" file.

We encourage you to try running the tutorial, but use one of the ```BioNet``` models generated in Chapter 3 in place of the ```PointNet``` model.

## Up Next

We have now shown the basic functionality of bmtk and SONATA, and how to use it to build a small toy model of the the mouse visual cortical system. In the next chapter, we will proceed to build a larger, more realistic model, and also demonstrate some of the more advanced features that are useful when trying to build and optimize a large to-scale model of a cortical circuit.


[**Proceed to Chapter 6**](../Ch6_l4model/6.%20More%20Realistic%20Model%20and%20Advanced%20Features.ipynb)